# Sample EDA of patent and SBIR dataframe
This is an incomplete notebook used for discovery purposes and to verify queries in more familiary pandas.

Think of this a scratch pad.

In [1]:
import pandas as pd
import requests
import io

## EDA Example 2:
Given a technology domain, find the set of companies working in that domain and determine if pairing them together would make sense?

### Get the SBIR entities data

In [2]:
# path to your SBIR entities file
df_sbir_abstract_entities = pd.read_csv('./preprocessed_files/neo4j_sbir_abstract_entities.csv')#, index=False)

In [3]:
print(len(df_sbir_abstract_entities))
#display(df_sbir_abstract_entities.head())
#df_sbir_abstract_entities.info()

20904


In [4]:
df_sbir_abstract_entities.head()

,key,Abstract,abstract_entities,entities
0,16068,Current automated RF signal acquisition and an...,"['condition', 'signal density', 'signal mingle...",condition
1,16068,Current automated RF signal acquisition and an...,"['condition', 'signal density', 'signal mingle...",signal density
2,16068,Current automated RF signal acquisition and an...,"['condition', 'signal density', 'signal mingle...",signal mingle
3,16068,Current automated RF signal acquisition and an...,"['condition', 'signal density', 'signal mingle...",signal intermodulation product desire
4,16068,Current automated RF signal acquisition and an...,"['condition', 'signal density', 'signal mingle...",Bandwidth ( ibw ) receiver wide bit-depth reduce


### Get the list of patents and their terms data

In [5]:
df_exploded_patents = pd.read_csv('./preprocessed_files/neo4j_patents_abstract.csv')#, index=False)

In [6]:
print(len(df_exploded_patents))
#display(df_exploded_patents.head())
#df_exploded_patents.info()

6612


In [7]:
df_exploded_patents.head(10)

,key,abstract,abstract_entities
0,20230227158,\nA system for performing work on electrical p...,splice electrical power line
1,20230227158,\nA system for performing work on electrical p...,power line tool
2,20230227158,\nA system for performing work on electrical p...,electrical power line
3,20230227158,\nA system for performing work on electrical p...,attachment point support
4,20230227158,\nA system for performing work on electrical p...,system
5,20230229051,\nThe disclosure relates generally to a method...,method
6,20230229051,\nThe disclosure relates generally to a method...,optical state base
7,20230229051,\nThe disclosure relates generally to a method...,switch
8,20230229051,\nThe disclosure relates generally to a method...,power
9,20230229051,\nThe disclosure relates generally to a method...,disclosure


### Define the domain with a seed of terms

In [8]:
key_terms = ['radar','signal','RFID']

In [9]:
#find the sbirs that contain at least one of the key_terms
sbir_company_seed_df = df_sbir_abstract_entities[['key','entities']][df_sbir_abstract_entities['entities'].isin(key_terms)]

In [69]:
#28 rows showing abstract matched to entities. Each abstract has only 1 or 2 key_terms
display(len(sbir_company_seed_df)) 
display(sbir_company_seed_df.sort_values(['key']))
display(sbir_company_seed_df.value_counts(['key']))

28

,key,entities
17536,585,signal
675,13447,radar
18803,31460,radar
18804,31460,signal
67,38328,signal
6730,49508,signal
8158,51264,signal
12003,54529,signal
1098,58313,RFID
6382,61389,signal


key   
31460     2
127109    2
128187    2
585       1
13447     1
51264     1
54529     1
38328     1
49508     1
61389     1
58313     1
69547     1
72872     1
84862     1
104573    1
73945     1
79804     1
113790    1
110645    1
121284    1
128485    1
139347    1
150382    1
157859    1
184591    1
Name: count, dtype: int64

In [80]:
#Now find all the entities of the give companies that have at least one of the key terms
sbir_terms_ds['entities'] = df_sbir_abstract_entities['entities'][df_sbir_abstract_entities['key'].isin(sbir_company_seed_df['key'])]

In [82]:
display(len(sbir_terms_ds))
sbir_terms_df = sbir_terms_ds.to_frame()
display(sbir_terms_df)

787

,entities
63,long range sigint
64,magnetic signature
65,electromagnetic sensing
66,low frequency electromagnetic
67,signal
...,...
18818,large system
18819,vehicle
18820,cost radar solution
18821,automotive radar


In [13]:
#find the patents that contain at least one of the key terms
pt_company_seed_df = df_exploded_patents[['key','abstract_entities']][df_exploded_patents['abstract_entities'].isin(key_terms)]

In [14]:
display(len(pt_company_seed_df))
display(pt_company_seed_df)

21

,key,abstract_entities
59,20230225236,signal
324,20230225267,signal
515,20230225293,signal
1102,20230225391,signal
2395,20230225570,signal
2540,20230225593,signal
2555,20230225594,signal
2623,20230225605,signal
2717,20230225614,signal
2817,20230225623,signal


In [15]:
pt_terms_df = df_exploded_patents['abstract_entities'][df_exploded_patents['key'].isin(pt_company_seed_df['key'])]
pt_terms_df.name = "entities"

In [16]:
display(len(pt_terms_df))
display(pt_terms_df)

242

58                              vehicle
59                               signal
60                     steering command
61                  cultivation process
62          row-detection device design
                     ...               
5056                      couple output
5057                  prosthetic device
5058    prosthetic device low extremity
5059                             device
5060                prosthetic food low
Name: entities, Length: 242, dtype: object

In [17]:
#Merge lists to create "domain", which is defined as the list of all these combined terms
domain = pd.merge(sbir_terms_df, pt_terms_df)
domain.drop_duplicates(inplace=True)
display(len(domain))
display(domain)

18

,entities
0,signal
483,detect
495,product
496,design
504,device
524,patient
536,system
542,distance
543,communication
544,information


In [18]:
#join the SBIR and PT data after filtering for only domain specific terms
sbir_domain_df = df_sbir_abstract_entities[['key','entities']][df_sbir_abstract_entities['entities'].isin(domain['entities'])]
pt_domain_df = df_exploded_patents[['key','abstract_entities']][df_exploded_patents['abstract_entities'].isin(domain['entities'])]

In [39]:
print(len(sbir_domain_df))
print(len(pt_domain_df))
pt_domain_df.loc[len(pt_domain_df.index)] = [20230227158, 'laben']

904
507


In [40]:
sbir_domain_df.rename(columns={'key': 'sbir_key'}, inplace=True)

In [41]:
pt_domain_df.rename(columns={'key': 'pt_key', 'abstract_entities': 'entities'}, inplace=True)

In [42]:
pt_domain_df

,pt_key,entities
4,20230227158,system
5,20230229051,method
58,20230225236,vehicle
59,20230225236,signal
63,20230225236,detect
...,...,...
6595,20230226142,method
6601,20230226143,patient
6606,20230226144,method
6608,20230226145,method


In [43]:
combine_df = pd.merge(sbir_domain_df, pt_domain_df, how='outer')
display(combine_df.info())
display(combine_df)
#combine_df.drop_duplicates()
#display(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30287 entries, 0 to 30286
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sbir_key  30286 non-null  float64
 1   entities  30287 non-null  object 
 2   pt_key    30287 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 710.0+ KB


None

,sbir_key,entities,pt_key
0,16068.0,condition,20230225259
1,16068.0,condition,20230225608
2,16068.0,condition,20230225615
3,16068.0,condition,20230225620
4,16068.0,condition,20230225632
...,...,...,...
30282,158100.0,distance,20230225807
30283,158100.0,distance,20230225856
30284,158100.0,distance,20230225883
30285,158100.0,distance,20230225921


In [44]:
count_series = combine_df.groupby(['sbir_key', 'pt_key']).size()
count_df = count_series.to_frame(name = 'size').reset_index()
display(count_df.head(20))

,sbir_key,pt_key,size
0,585.0,20230225236,2
1,585.0,20230225254,1
2,585.0,20230225267,1
3,585.0,20230225286,1
4,585.0,20230225288,1
5,585.0,20230225293,2
6,585.0,20230225294,1
7,585.0,20230225295,1
8,585.0,20230225303,1
9,585.0,20230225305,1


In [45]:
#The number of times the sibr and pt have the same entity
c2 = combine_df.value_counts(['sbir_key', 'pt_key']).to_frame('counts').reset_index()
c2

,sbir_key,pt_key,counts
0,117983.0,20230225570,8
1,117983.0,20230225501,4
2,117983.0,20230225502,4
3,117983.0,20230225524,4
4,117983.0,20230225542,4
...,...,...,...
28869,57716.0,20230225820,1
28870,57716.0,20230225814,1
28871,57716.0,20230225813,1
28872,57716.0,20230225805,1


In [35]:
s2 = sbir_domain_df.value_counts(['sbir_key']).to_frame('counts').reset_index()
s2

,sbir_key,counts
0,117983,8
1,51264,8
2,111367,6
3,72872,6
4,23381,6
...,...,...
549,17735,1
550,15107,1
551,16475,1
552,147816,1


In [47]:
s_combine_df = pd.merge(sbir_domain_df, sbir_domain_df, how='outer', on=['entities'])
display(s_combine_df.info())
display(s_combine_df)
#combine_df.drop_duplicates()
#display(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81396 entries, 0 to 81395
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sbir_key_x  81396 non-null  int64 
 1   entities    81396 non-null  object
 2   sbir_key_y  81396 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


None

,sbir_key_x,entities,sbir_key_y
0,16068,condition,16068
1,16068,condition,69517
2,16068,condition,98560
3,16068,condition,46965
4,16068,condition,173286
...,...,...,...
81391,158100,distance,51264
81392,158100,distance,106617
81393,158100,distance,97362
81394,158100,distance,30984


## Playing around with simple test data to test logic

In [53]:
d = {'SP1': ['O1', 'O1'], 'Term': ['A', 'B']}
sp1_df = pd.DataFrame(data=d)
display(sp1_df)

,SP1,Term
0,O1,A
1,O1,B


In [59]:
d2 = {'SP2': ['x1', 'X1'], 'Term': ['A', 'C']}
#d2 = {'SP2': ['x1'], 'Term': ['A']}
sp2_df = pd.DataFrame(data=d2)
display(sp2_df)

,SP2,Term
0,x1,A
1,X1,C


In [60]:
csp_df = pd.merge(sp1_df, sp2_df, how='outer', on=['Term'])
csp_df

,SP1,Term,SP2
0,O1,A,x1
1,O1,B,NaN
2,NaN,C,X1


#store the equivalent NEO4J query progress:
#8.Given a technology domain, find the set of companies working in that domain and determine if pairing them together would make sense?

#Find all the entities that is one of the key words
#Find all the SBIRS and PATENTS that contain at least one of those entities
#Find all the entities those SBIRS and PATENTS contain and combine that into a list called terms
#Find which combination of two (SBIR, PATENTS) contains the most number of the terms
#Refinement, could also order them by number of overlapping words and the pair with least overlap could be the best partner...
#present this list of a human as a starting point to analys further for a client

:param key_words => ['radar','signal','RFID']
:param key_words => ['computer','GPU','CPU','memory','processing','hard drive']

#This query will get the complete list of terms that define the domain
CALL {
    MATCH (e1:Entities)-[]-(:Sbir_Abstract|Patent_Abstract)-[]-(e2:Entities)
    WHERE e1.entities IN $key_words
    RETURN e2.entities AS terms
    UNION
    MATCH (e1:Entities)
    WHERE e1.entities IN $key_words
    RETURN e1.entities AS terms
    ORDER BY terms
}
RETURN DISTINCT terms
ORDER BY terms

#This query will rank the combinations of two companies that best cover the domain
CALL {
    MATCH (e1:Entities)-[]-(:Sbir_Abstract|Patent_Abstract)-[]-(e2:Entities)
    WHERE e1.entities IN $key_words
    RETURN e2.entities AS terms
    UNION
    MATCH (e1:Entities)
    WHERE e1.entities IN $key_words
    RETURN e1.entities AS terms
    ORDER BY terms
}
WITH DISTINCT terms
MATCH (sp:Sbir_Abstract|Patent_Abstract)-[]-(e:Entities)
WHERE e.entities in terms
WITH sp.key as abstract1, count(DISTINCT e) as num_terms1, collect(e) as terms1
MATCH (sp2:Sbir_Abstract|Patent_Abstract)-[]-(e2:Entities)
WHERE e.entities in (terms1)

WHERE num_terms1 >= size($key_words)
RETURN abstract1, abstract2, num_terms1, combined_terms
ORDER BY num_terms1 DESC

CALL {
    MATCH (e1:Entities)-[]-(:Sbir_Abstract|Patent_Abstract)-[]-(e2:Entities)
    WHERE e1.entities IN $key_words
    RETURN e2.entities AS terms
    UNION
    MATCH (e1:Entities)
    WHERE e1.entities IN $key_words
    RETURN e1.entities AS terms
    ORDER BY terms
}
WITH DISTINCT terms
MATCH (sp:Sbir_Abstract|Patent_Abstract)-[]-(e:Entities)-[]-(sp2:Sbir_Abstract|Patent_Abstract)
WHERE e.entities in terms
WITH sp, sp2, count(DISTINCT e) as num_terms1, collect(e.entities) as combined_terms
MATCH (p:Principal)-[]-(sp)-[]-(e:Entities), (p2:Principal)-[]-(sp2)-[]-(e:Entities)
WHERE e.Enitities IN combined_terms AND num_terms1 >= size($key_words)
RETURN p.Company as candidate1, p2.Company as candidate2, num_terms1, combined_terms
ORDER BY num_terms1 DESC

WITH sp.key as candidate1, sp2.key as candidate2, count(DISTINCT e) as num_terms1, collect(e.entities) as combined_terms
